In [1]:
import langchain
import os
import warnings
warnings.filterwarnings('ignore')

# 환경변수 세팅

os.environ['OPENAI_API_KEY'] = '8c91c5c7058847f3820f9d7c12bbb1fb'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://sktfly-ai.openai.azure.com/'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

In [2]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')
tiktoken_len = lambda x:len(tokenizer.encode(x))


In [3]:
from langchain.text_splitter import  RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=130,
    chunk_overlap=50,
    length_function=tiktoken_len,
)

from langchain.document_loaders import  PyPDFLoader

loader = PyPDFLoader('data/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf')
pages = loader.load_and_split()
print(len(pages))



19


In [4]:
from langchain.document_loaders import PyPDFLoader

loader=PyPDFLoader('data/K-패스 국토교통부 자료.pdf')
pages=loader.load_and_split()

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':True}

hf = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

## Vector DB: Chroma

In [14]:
!pip install chromadb -q

In [6]:
from langchain.vectorstores import Chroma

docs = text_splitter.split_documents(pages)

db = Chroma.from_documents(docs, hf)

In [9]:
query = 'K-패스'
docs_result = db.similarity_search(query) # 유사도 검색

print(docs_result[0].page_content) # 가장 유사한 페이지 
print('-' * 100)
print(docs_result[0].metadata['page']) # 페이지 번호 

상품을 골라 패스를 이용할 수 있다 K- .
    * 신한 하나 우리 현대 삼성 국민 농협 티머니 이동의즐거움 유페이 , , , , , BC, KB , NH , , , DGB
----------------------------------------------------------------------------------------------------
1


In [10]:
docs_result = db.similarity_search_with_relevance_scores(query, k=3)
print(len(docs_result))
docs_result

3


[(Document(page_content='상품을 골라 패스를 이용할 수 있다 K- .\n    * 신한 하나 우리 현대 삼성 국민 농협 티머니 이동의즐거움 유페이 , , , , , BC, KB , NH , , , DGB', metadata={'page': 1, 'source': 'data/K-패스 국토교통부 자료.pdf'}),
  0.36063458401955717),
 (Document(page_content='- 3 -참고1 패스 사업 개요 K-\n□ (사업명칭 대중교통비 환급 지원 사업) 가칭 ( 패스K-)\n□ (사업방식 알뜰교통카드의 ) 이동거리 기준 적립 방식을 없애고 ,', metadata={'page': 2, 'source': 'data/K-패스 국토교통부 자료.pdf'}),
  0.3407083126868796),
 (Document(page_content='빈틈없이 이행하여 월에 패스를 차질없이 출시하고 각 지자체와 5 K- , \n협력하여 패스를 기반으로 지역민 혜택은 더욱 강화할 수 있는 K- 지역\n맞춤형 패스 사업K- 도 적극 발굴 해나갈 계획 이라고 밝혔다 ” .', metadata={'page': 1, 'source': 'data/K-패스 국토교통부 자료.pdf'}),
  0.3310901911224511)]

In [11]:
db = Chroma.from_documents(docs, hf, persist_directory='./chroma_db') # DB 저장

In [13]:
db2 = Chroma(persist_directory='./chroma_db', embedding_function=hf) # DB 로드
docs_result = db2.similarity_search(query) # 유사도 검색
docs_result

[Document(page_content='상품을 골라 패스를 이용할 수 있다 K- .\n    * 신한 하나 우리 현대 삼성 국민 농협 티머니 이동의즐거움 유페이 , , , , , BC, KB , NH , , , DGB', metadata={'page': 1, 'source': 'data/K-패스 국토교통부 자료.pdf'}),
 Document(page_content='상품을 골라 패스를 이용할 수 있다 K- .\n    * 신한 하나 우리 현대 삼성 국민 농협 티머니 이동의즐거움 유페이 , , , , , BC, KB , NH , , , DGB', metadata={'page': 1, 'source': 'data/K-패스 국토교통부 자료.pdf'}),
 Document(page_content='- 3 -참고1 패스 사업 개요 K-\n□ (사업명칭 대중교통비 환급 지원 사업) 가칭 ( 패스K-)\n□ (사업방식 알뜰교통카드의 ) 이동거리 기준 적립 방식을 없애고 ,', metadata={'page': 2, 'source': 'data/K-패스 국토교통부 자료.pdf'}),
 Document(page_content='- 3 -참고1 패스 사업 개요 K-\n□ (사업명칭 대중교통비 환급 지원 사업) 가칭 ( 패스K-)\n□ (사업방식 알뜰교통카드의 ) 이동거리 기준 적립 방식을 없애고 ,', metadata={'page': 2, 'source': 'data/K-패스 국토교통부 자료.pdf'})]

## Vector DB: FAISS

In [14]:
!pip install faiss-cpu -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, hf)
docs_result = db.similarity_search(query)
docs_result

'Run more documents through the embeddings and add to the vectorstore.\n\n        Args:\n            documents (List[Document]: Documents to add to the vectorstore.\n\n        Returns:\n            List[str]: List of IDs of the added texts.\n        '